In [54]:
pip install selenium

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.7 MB 5.6 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/9.7 MB 7.3 MB/s eta 0:00:01
   -------------------- ------------------- 5.0/9.7 MB 8.2 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.7 MB 8.7 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.7 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 8.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [57]:
pip install webdriver-manager


In [1]:
from bs4 import BeautifulSoup
import requests
from requests import get
import time
import random
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import os

### Вариант с переходом на страницу объявления

In [ ]:

URL_TEMPLATE = 'https://auto.ru/cars/all/?page='
PAUSE_DURATION_SECONDS = 4
PAUSE_DURATION_DESCR_SECONDS = 0.5
MAX_PAGES = 99  # Количество страниц для парсинга

def parse_ad_details(driver):
    """Функция для парсинга деталей объявления, включая описание, дату и ID."""
    try:
        # Ожидаем появления блока с описанием объявления
        WebDriverWait(driver, PAUSE_DURATION_DESCR_SECONDS).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'CardDescription__text'))
        )
        # Парсинг описания
        description_div = driver.find_element(By.CLASS_NAME, 'CardDescription__text')
        description_texts = [span.text.strip().replace("\n", ", ") for span in description_div.find_elements(By.TAG_NAME, 'span')]
        ad_description = ', '.join(description_texts)
    except TimeoutException:
        ad_description = None

    # Парсинг даты размещения объявления
    try:
        ad_date_div = driver.find_element(By.CLASS_NAME, 'CardHead__creationDate')
        ad_date = ad_date_div.text.strip()
    except:
        ad_date = None

    # Парсинг ID объявления
    try:
        ad_id_div = driver.find_element(By.CLASS_NAME, 'CardHead__id')
        ad_id = ad_id_div.text.strip()
    except:
        ad_id = None

    return ad_description, ad_date, ad_id


def parse_page(driver, soup):
    car_blocks = soup.find_all('div', class_='ListingItem__main')

    names = []
    descriptions = []
    prices = []
    ad_descriptions = []
    cities = []
    parsing_times = []
    ad_paths = []
    years = []
    probegs = []
    ad_dates = []
    ad_ids = []

    for car in car_blocks:
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        parsing_times.append(current_time)
        
        # Парсинг названия
        try:
            name_tag = car.find('a', {"class": "Link ListingItemTitle__link"})
            name = name_tag.text.strip() if name_tag else None
        except:
            name = None

        # Парсинг описания автомобиля
        try:
            description_tag = car.find_all('div', {"class": "ListingItemTechSummaryDesktop__cell"})
            if description_tag:
                description = ', '.join([tag.get_text(strip=True).replace("/", ",") for tag in description_tag])
            else:
                description = np.nan
        except:
            description = np.nan

        # Парсинг цены
        try:
            price_tags = car.find_all('div', {"class": "ListingItem__priceBlock"})
            price = ', '.join([tag.get_text(strip=True).replace("\u00a0", " ").replace("Купить в кредит", "") for tag in price_tags]) if price_tags else None
        except:
            price = None

        # Парсинг года и пробега
        try:
            year_tag = car.find('div', {"class": "ListingItem__year"})
            year = year_tag.text.strip() if year_tag else None
        except:
            year = None
       
        try:
            probeg_tag = car.find('div', {"class": "ListingItem__kmAge"})
            probeg = probeg_tag.text.strip() if probeg_tag else None
        except:
            probeg = None

        # Парсинг города
        try:
            city_tag = car.find('span', {"class": "MetroListPlace__regionName MetroListPlace_nbsp"})
            city = city_tag.text.strip() if city_tag else None
        except:
            city = None

        # Парсинг ссылки объявления
        try:
            ad_path_tag = car.find('a', {"class": "Link ListingItemTitle__link"})
            ad_path = ad_path_tag['href'].strip() if ad_path_tag and 'href' in ad_path_tag.attrs else None
        except:
            ad_path = None

        # Переход по ссылке объявления и парсинг описания, даты и ID объявления
        if ad_path:
            driver.get(ad_path)
            time.sleep(PAUSE_DURATION_DESCR_SECONDS)
            ad_description, ad_date, ad_id = parse_ad_details(driver)
        else:
            ad_description = ad_date = ad_id = None

        # Добавляем данные в списки
        names.append(name)
        prices.append(price)
        ad_descriptions.append(ad_description)
        cities.append(city)
        ad_paths.append(ad_path)
        years.append(year)
        probegs.append(probeg)
        ad_dates.append(ad_date)
        ad_ids.append(ad_id)
        descriptions.append(description)
        

    # Создаем DataFrame
    df_page = pd.DataFrame({
        'name': names,
        'price': prices,
        'description': descriptions,
        'year': years,
        'ad_description': ad_descriptions,
        'probeg': probegs,
        'city': cities,
        'parsing_time': parsing_times,
        'ad_path': ad_paths,
        'ad_date': ad_dates,
        'ad_id': ad_ids
    })

    return df_page


def main():
    all_cars_df = pd.DataFrame()

    # Проверяем, существует ли файл с уже спарсенными данными
    if os.path.exists('cars_data_autoru.csv'):
        all_cars_df = pd.read_csv('cars_data_autoru.csv')

    for page_num in range(1, MAX_PAGES + 1):
        print(f"Парсим страницу {page_num}...")

        try:
            url = URL_TEMPLATE + str(page_num)
            driver.get(url)
            time.sleep(PAUSE_DURATION_SECONDS)

            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            # Парсим данные на текущей странице и добавляем их в DataFrame
            df_page = parse_page(driver, soup)
            all_cars_df = pd.concat([all_cars_df, df_page], ignore_index=True)

            # Сохраняем промежуточный результат
            all_cars_df.to_csv('cars_data_autoru.csv', index=False, encoding='utf-8-sig')

        except TimeoutException:
            print(f"Страница {page_num} не загрузилась, пропускаем...")
            continue
        except Exception as e:
            print(f"Ошибка на странице {page_num}: {e}")
            continue

    print("Парсинг завершен.")
    print(all_cars_df.head())
    all_cars_df.to_csv('cars_data_autoru.csv', index=False, encoding='utf-8-sig')

if __name__ == '__main__':
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        main()
    except Exception as e:
        print(e)
    finally:
        driver.quit()


Парсим страницу 1...
Описание не загрузилось.
Описание не загрузилось.
Парсим страницу 2...
Парсим страницу 3...
Описание не загрузилось.
Парсим страницу 4...
Парсим страницу 5...


In [ ]:
# преобразование видимости датафрейма
pd.set_option('display.max_colwidth', None)

pd.set_option('display.width', 200)

In [ ]:
# Теперь сбрасываем эти опции если надо
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')


In [ ]:
df_cars = pd.read_csv('cars_data_autoru.csv')
df_cars.head()